### Задание 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('C:\\Users\\Dmitry\\Documents\\Учеба\\3 Python и математика для анализа данных\\3.7 Pandas функции и группировки\\ml-latest-small\\ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
# считаю количество оценок, поставленных каждым пользователем
ratings_grouped = ratings.groupby('userId').count().reset_index()
# отфильтровываю пользователе, которые выставили более 100 оценок
ratings_filtered = ratings_grouped[ratings_grouped.rating > 100]
# беру их Id
filtered_id = ratings_filtered['userId'].tolist()
# исходная таблица только с пользователями, поставившими более 100 оценок
ratings_tot = ratings[ratings['userId'].isin(filtered_id)]
ratings_tot.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [4]:
# дата первой оценки
min_ts = ratings_tot.groupby('userId').min().reset_index()[['userId', 'timestamp']]
min_ts.rename(columns = {'timestamp': 'min_timestamp'}, inplace = True)
min_ts.head()
#min_ts.shape

,userId,min_timestamp
0,4,949778714
1,8,1154389340
2,15,997937239
3,17,1127468587
4,19,855190091


In [5]:
# дата последней оценки
max_ts = ratings_tot.groupby('userId').max().reset_index()[['userId', 'timestamp']]
max_ts.rename(columns = {'timestamp': 'max_timestamp'}, inplace = True)
max_ts.head()
#max_ts.shape

,userId,max_timestamp
0,4,949982274
1,8,1154474527
2,15,1469330735
3,17,1127476640
4,19,855195373


In [6]:
joined_ts = max_ts.merge(min_ts, how = 'left')
joined_ts.head()

,userId,max_timestamp,min_timestamp
0,4,949982274,949778714
1,8,1154474527,1154389340
2,15,1469330735,997937239
3,17,1127476640,1127468587
4,19,855195373,855190091


In [7]:
def lt(ts):
    return ts['max_timestamp'] - ts['min_timestamp']

In [8]:
joined_ts['LT'] = joined_ts.apply(lt, axis = 1)
joined_ts.head()

,userId,max_timestamp,min_timestamp,LT
0,4,949982274,949778714,203560
1,8,1154474527,1154389340,85187
2,15,1469330735,997937239,471393496
3,17,1127476640,1127468587,8053
4,19,855195373,855190091,5282


In [20]:
lt = joined_ts['LT'].mean() / 3600 / 24
print(f"Среднее время жизни пользователя {round(lt, 0)} дня")

Среднее время жизни пользователя 464.0 дня


### Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [10]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [13]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [14]:
joined = client_base.merge(rzd, how = 'outer').merge(air, how = 'outer').merge(auto, how = 'outer')[['client_id',\
                                                            'rzd_revenue','air_revenue', 'auto_revenue']]
joined

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,57483.0
3,114,5774.0,NaN,83.0
4,115,981.0,81.0,912.0
5,116,NaN,4.0,4834.0
6,117,NaN,13.0,98.0
7,118,NaN,173.0,NaN


In [15]:
joined_full = client_base.merge(rzd, how = 'outer').merge(air, how = 'outer').merge(auto, how = 'outer')
joined_full

,client_id,address,rzd_revenue,air_revenue,auto_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,NaN,57483.0
3,114,Мира 14,5774.0,NaN,83.0
4,115,ЗЖБИиДК 1,981.0,81.0,912.0
5,116,Строителей 18,NaN,4.0,4834.0
6,117,Панфиловская 33,NaN,13.0,98.0
7,118,Мастеркова 4,NaN,173.0,NaN


### Задача 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Я объединил бы эти данные с таблицами визитов и покупок через user_id и написал бы функцию, которая в зависимоти от координат возвращает название города (страны, региона или района в зависимости от специфики бизнеса) откуда были совершены визиты. Потом с помощью группировок полученных результатов можно определить откуда приходят клиенты или откуда они не приходят и, таким образом, найти точки для усиления.
P.S. конечно, для того, чтобы написать такую функцию потребуется база с координатами, но, мне кажется, что такие уже есть.